In [66]:
import nltk
from nltk.corpus import stopwords
from nltk import SnowballStemmer

In [67]:
ppdbDict = {}
ppdbSim = 0.9
theta1 = 0.9
punctuations = ['(','-lrb-','.',',','-','?','!',';','_',':','{','}','[','/',']','...','"','\'',')', '-rrb-']
stemmer = SnowballStemmer('english')
punctuations = ['(','-lrb-','.',',','-','?','!',';','_',':','{','}','[','/',']','...','"','\'',')', '-rrb-']
stopwords = stopwords.words('english')

In [73]:
def loadPPDB(ppdbFileName = 'monolingual-word-aligner/Resources/ppdb-1.0-xxxl-lexical.extended.synonyms.uniquepairs'):

    global ppdbSim
    global ppdbDict

    count = 0
    
    ppdbFile = open(ppdbFileName, 'r')
    for line in ppdbFile:
        if line == '\n':
            continue
        tokens = line.split()
        tokens[1] = tokens[1].strip()
        ppdbDict[(tokens[0], tokens[1])] = ppdbSim
        count += 1

        
def presentInPPDB(word1, word2):

    global ppdbDict

    if (word1.lower(), word2.lower()) in ppdbDict:
        return True
    if (word2.lower(), word1.lower()) in ppdbDict:
        return True

def wordRelatedness(word1, pos1, word2, pos2):

    global stemmer
    global ppdbSim
    global punctuations

    if len(word1) > 1:
        canonicalWord1 = word1.replace('.', '')
        canonicalWord1 = canonicalWord1.replace('-', '')
        canonicalWord1 = canonicalWord1.replace(',', '')
    else:
        canonicalWord1 = word1
        
    if len(word2) > 1:
        canonicalWord2 = word2.replace('.', '')
        canonicalWord2 = canonicalWord2.replace('-', '')
        canonicalWord2 = canonicalWord2.replace(',', '')
    else:
        canonicalWord2 = word2
    
    
    if canonicalWord1.lower() == canonicalWord2.lower():
        return 1

    if stemmer.stem(word1).lower() == stemmer.stem(word2).lower():
        return 1

    if canonicalWord1.isdigit() and canonicalWord2.isdigit() and canonicalWord1 <> canonicalWord2:
        return 0

    if pos1.lower() == 'cd' and pos2.lower() == 'cd' and (not canonicalWord1.isdigit() and not canonicalWord2.isdigit()) and canonicalWord1 <> canonicalWord2:
        return 0

    # stopwords can be similar to only stopwords
    if (word1.lower() in stopwords and word2.lower() not in stopwords) or (word1.lower() not in stopwords and word2.lower() in stopwords):
        return 0

    # punctuations can only be either identical or totally dissimilar
    if word1 in punctuations or word2 in punctuations:
        return 0

    if presentInPPDB(word1.lower(), word2.lower()):
        return ppdbSim
    else:
        return 0


In [69]:
loadPPDB()

In [99]:
"4".isdigit()

True

In [107]:
def findContinuousSublists(S1_tokens, S2_tokens, use_lower=True):
    s1 = []
    s2 = []
    if(use_lower):
        S1_tokens = map(lambda token: token.lower(), S1_tokens)
        S2_tokens = map(lambda token: token.lower(), S2_tokens)
    for token in S1_tokens:
        s1.append(token)
    for token in S2_tokens:
        s2.append(token)
    
    s1_size = len(s1)
    commonsublists = []
    for size in xrange(s1_size,0,-1):
        starting_indices_S1 = [item for item in xrange(0,len(s1)-size+1)]
        starting_indices_S2 = [item for item in xrange(0,len(s2)-size+1)]
        for i in starting_indices_S1:
            for j in starting_indices_S2:
                if(s1[i:i+size]==s2[j:j+size]):
                    alreadyInserted = False
                    currentS1_indices = [ind for ind in xrange(i,i+size)]
                    currentS2_indices = [ind for ind in xrange(j,j+size)]
                    for item in commonsublists:
                        if(isSublist(currentS1_indices,item[0]) and isSublist(currentS2_indices, item[1])):
                            alreadyInserted = True
                            break;
                    if not alreadyInserted:
                        commonsublists.append([currentS1_indices, currentS2_indices])
    return commonsublists

def isSublist(A, B):
# returns True if A is a sublist of B, False otherwise
    sub = True

    for item in A:
        if item not in B:
            sub = False
            break
    
    return sub


In [84]:
import coreNlpUtil

In [85]:
from coreNlpUtil import *

In [95]:
def findAllCommonContiguousSublists(A, B, turnToLowerCases=True): # this is a very inefficient implementation, you can use suffix trees to devise a much faster method
# returns all the contiguous sublists in order of decreasing length
# output format (0-indexed):
# [
#    [[indices in 'A' for common sublist 1], [indices in 'B' for common sublist 1]],
#    ...,
#    [[indices in 'A' for common sublist n], [indices in 'B' for common sublist n]]
# ]

    a = []
    b = []
    for item in A:
        a.append(item)
    for item in B:
        b.append(item)

    if turnToLowerCases:
        for i in xrange(len(a)):
            a[i] = a[i].lower()
        for i in xrange(len(b)):
            b[i] = b[i].lower()
            

    commonContiguousSublists = []

    swapped = False
    if len(a) > len(b):
        temp = a
        a = b
        b = temp
        swapped = True

    maxSize = len(a)
    for size in xrange(maxSize, 0, -1):
        startingIndicesForA = [item for item in xrange(0, len(a)-size+1)]
        startingIndicesForB = [item for item in xrange(0, len(b)-size+1)]
        for i in startingIndicesForA:
            for j in startingIndicesForB:
                if a[i:i+size] == b[j:j+size]:
                    # check if a contiguous superset has already been inserted; don't insert this one in that case
                    alreadyInserted = False
                    currentAIndices = [item for item in xrange(i,i+size)]
                    currentBIndices = [item for item in xrange(j,j+size)]
                    for item in commonContiguousSublists:
                        if isSublist(currentAIndices, item[0]) and isSublist(currentBIndices, item[1]):
                            alreadyInserted = True
                            break
                    if not alreadyInserted:
                        commonContiguousSublists.append([currentAIndices, currentBIndices])
    
    
    

    if swapped:
        for item in commonContiguousSublists:
            temp = item[0]
            item[0] = item[1]
            item[1] = temp

                        
    return commonContiguousSublists

def isSublist(A, B):
# returns True if A is a sublist of B, False otherwise
    sub = True

    for item in A:
        if item not in B:
            sub = False
            break
    
    return sub

In [151]:
s1 = ["check","this","Coke","out","isnt","this","wonderful"]
s2 = ["test","this","pepsi","out","dude","isn't","this","wonderful"]

In [152]:
findAllCommonContiguousSublists(["check","this","out","isnt","this","wonderful"], ["test","this","out","dude","isn't","this","wonderful"], turnToLowerCases=True)

[[[1, 2], [1, 2]], [[4, 5], [5, 6]], [[1], [5]], [[4], [1]]]

In [218]:
sentence1 = "check this Daniel Sam out isnt this wonderful Indian Ocean"
sentence2 = "test this Dany out dude isn't this wonderful India"
if isinstance(sentence1, list):
    sentence1 = ' '.join(sentence1)
if isinstance(sentence2, list):
    sentence2 = ' '.joinsentence1ParseResult(sentence2)

sentence1ParseResult = parseText(sentence1)
sentence2ParseResult = parseText(sentence2)

sentence1Lemmatized = lemmatize(sentence1ParseResult)
sentence2Lemmatized = lemmatize(sentence2ParseResult)

sentence1PosTagged = posTag(sentence1ParseResult)
sentence2PosTagged = posTag(sentence2ParseResult)
sentence1LemmasAndPosTags = []

for i in xrange(len(sentence1Lemmatized)):
    sentence1LemmasAndPosTags.append([])    
for i in xrange(len(sentence1Lemmatized)):
    for item in sentence1Lemmatized[i]:
        sentence1LemmasAndPosTags[i].append(item)
    sentence1LemmasAndPosTags[i].append(sentence1PosTagged[i][3])

sentence2LemmasAndPosTags = []
for i in xrange(len(sentence2Lemmatized)):
    sentence2LemmasAndPosTags.append([])    
for i in xrange(len(sentence2Lemmatized)):
    for item in sentence2Lemmatized[i]:
        sentence2LemmasAndPosTags[i].append(item)
    sentence2LemmasAndPosTags[i].append(sentence2PosTagged[i][3])

    


In [219]:
source = sentence1LemmasAndPosTags 
target = sentence2LemmasAndPosTags
sourceWordIndices = [i+1 for i in xrange(len(source))]
targetWordIndices = [i+1 for i in xrange(len(target))]


alignments = []
sourceWordIndicesAlreadyAligned= []
targetWordIndicesAlreadyAligned= []

sourceWords = [item[2] for item in source]
targetWords = [item[2] for item in target]

sourceLemmas = [item[3] for item in source]
targetLemmas = [item[3] for item in target]

sourcePosTags = [item[4] for item in source]
targetPosTags = [item[4] for item in target]
commonContiguousSublists  = findAllCommonContiguousSublists(sourceWords, targetWords, True)
print commonContiguousSublists


[[[6, 7], [7, 8]], [[1], [1]], [[1], [7]], [[4], [3]], [[6], [1]]]


In [220]:

commonContiguousSublists  = findContinuousSublists(sourceWords, targetWords, True)
for item in commonContiguousSublists:
    if len(item[0]) >= 2:
        for j in xrange(len(item[0])):
            if item[0][j]+1 not in sourceWordIndicesAlreadyAligned and item[1][j]+1 not in targetWordIndicesAlreadyAligned and [item[0][j]+1, item[1][j]+1] not in alignments:
                alignments.append([item[0][j]+1, item[1][j]+1])
                sourceWordIndicesAlreadyAligned.append(item[0][j]+1)
                targetWordIndicesAlreadyAligned.append(item[1][j]+1)


In [207]:
sourceNamedEntities = ner(sentence1ParseResult)
sourceNamedEntities = sorted(sourceNamedEntities, key=len)
targetNamedEntities = ner(sentence2ParseResult)
targetNamedEntities = sorted(targetNamedEntities, key=len)


In [221]:
print alignments


[[7, 8], [8, 9]]


In [243]:
def align_Named_Entities(source, target, sourceParseResult, targetParseResult, existingAlignments):
    
    global punctuations

    alignments = []

    sourceNamedEntities = ner(sourceParseResult)
    sourceNamedEntities = sorted(sourceNamedEntities, key=len)
    
    targetNamedEntities = ner(targetParseResult)
    targetNamedEntities = sorted(targetNamedEntities, key=len)
    

    sourceNamedEntitiesAlreadyAligned = []
    targetNamedEntitiesAlreadyAligned = []


    # align all full matches
    for item in sourceNamedEntities:
        if item[3] not in ['PERSON', 'ORGANIZATION', 'LOCATION']:
            continue

        for jtem in targetNamedEntities:
            print jtem
            if jtem[3] not in ['PERSON', 'ORGANIZATION', 'LOCATION']:
                continue

            # get rid of dots and hyphens
            canonicalItemWord = [i.replace('.', '') for i in item[2]]
            canonicalItemWord = [i.replace('-', '') for i in item[2]]
            canonicalJtemWord = [j.replace('.', '') for j in jtem[2]]
            canonicalJtemWord = [j.replace('-', '') for j in jtem[2]]

            if canonicalItemWord == canonicalJtemWord:
                for k in xrange(len(item[1])):
                    print k
                    if ([item[1][k], jtem[1][k]]) not in alignments:
                        alignments.append([item[1][k], jtem[1][k]])
                sourceNamedEntitiesAlreadyAligned.append(item)
                targetNamedEntitiesAlreadyAligned.append(jtem)
    
    
    # align subset matches
    for item in sourceNamedEntities:
        if item[3] not in ['PERSON', 'ORGANIZATION', 'LOCATION'] or item in sourceNamedEntitiesAlreadyAligned:
            continue



        for jtem in targetNamedEntities:
            
            # find if the first is a part of the second
            print item, jtem
            print item[2], jtem[2]
            if isSublist(item[2], jtem[2]):
                unalignedWordIndicesInTheLongerName = []
                for ktem in jtem[1]:
                    unalignedWordIndicesInTheLongerName.append(ktem)
                for k in xrange(len(item[2])):
                    for l in xrange(len(jtem[2])):
                        if item[2][k] == jtem[2][l] and [item[1][k], jtem[1][l]] not in alignments:
                            alignments.append([item[1][k], jtem[1][l]])
                            if jtem[1][l] in unalignedWordIndicesInTheLongerName:
                                unalignedWordIndicesInTheLongerName.remove(jtem[1][l])
                for k in xrange(len(item[1])): # the shorter name
                    for l in xrange(len(jtem[1])): # the longer name
                        # find if the current term in the longer name has already been aligned (before calling alignNamedEntities()), do not align it in that case
                        alreadyInserted = False
                        for mtem in existingAlignments:
                            if mtem[1] == jtem[1][l]:
                                alreadyInserted = True
                                break
                        if jtem[1][l] not in unalignedWordIndicesInTheLongerName or alreadyInserted:
                            continue
                        if [item[1][k], jtem[1][l]] not in alignments  and target[jtem[1][l]-1][2] not in sourceWords  and item[2][k] not in punctuations and jtem[2][l] not in punctuations:
                            alignments.append([item[1][k], jtem[1][l]])
                
            # else find if the second is a part of the first
            elif isSublist(jtem[2], item[2]):
                unalignedWordIndicesInTheLongerName = []
                for ktem in item[1]:
                    unalignedWordIndicesInTheLongerName.append(ktem)
                for k in xrange(len(jtem[2])):
                    for l in xrange(len(item[2])):
                        if jtem[2][k] == item[2][l] and [item[1][l], jtem[1][k]] not in alignments:
                            alignments.append([item[1][l], jtem[1][k]])
                            if item[1][l] in unalignedWordIndicesInTheLongerName:
                                unalignedWordIndicesInTheLongerName.remove(item[1][l])
                for k in xrange(len(jtem[1])): # the shorter name
                    for l in xrange(len(item[1])): # the longer name
                        # find if the current term in the longer name has already been aligned (before calling alignNamedEntities()), do not align it in that case
                        alreadyInserted = False
                        for mtem in existingAlignments:
                            print mtem
                            if mtem[0] == item[1][k]:
                                alreadyInserted = True
                                break
                        if item[1][l] not in unalignedWordIndicesInTheLongerName or alreadyInserted:
                            continue
                        if [item[1][l], jtem[1][k]] not in alignments  and source[item[1][k]-1][2] not in targetWords  and item[2][l] not in punctuations and jtem[2][k] not in punctuations:
                            alignments.append([item[1][l], jtem[1][k]])
                            #unalignedWordIndicesInTheLongerName.remove(jtem[1][l])




In [244]:
print align_Named_Entities(sentence1LemmasAndPosTags, sentence2LemmasAndPosTags, sentence1ParseResult, sentence2ParseResult, alignments)

[[[u'10', u'14']], [3], [u'Dany'], u'PERSON']
[[[u'45', u'50']], [10], [u'India'], u'LOCATION']
[[[u'10', u'14']], [3], [u'Dany'], u'PERSON']
[[[u'45', u'50']], [10], [u'India'], u'LOCATION']
[[[u'11', u'17'], [u'18', u'21']], [3, 4], [u'Daniel', u'Sam'], u'PERSON'] [[[u'10', u'14']], [3], [u'Dany'], u'PERSON']
[u'Daniel', u'Sam'] [u'Dany']
[[[u'11', u'17'], [u'18', u'21']], [3, 4], [u'Daniel', u'Sam'], u'PERSON'] [[[u'45', u'50']], [10], [u'India'], u'LOCATION']
[u'Daniel', u'Sam'] [u'India']
[[[u'46', u'52'], [u'53', u'58']], [9, 10], [u'Indian', u'Ocean'], u'LOCATION'] [[[u'10', u'14']], [3], [u'Dany'], u'PERSON']
[u'Indian', u'Ocean'] [u'Dany']
[[[u'46', u'52'], [u'53', u'58']], [9, 10], [u'Indian', u'Ocean'], u'LOCATION'] [[[u'45', u'50']], [10], [u'India'], u'LOCATION']
[u'Indian', u'Ocean'] [u'India']
None


In [223]:
def alignNamedEntities(source, target, sourceParseResult, targetParseResult, existingAlignments):
# source and target:: each is a list of elements of the form:
# [[character begin offset, character end offset], word index, word, lemma, pos tag]

    global punctuations

    alignments = []

    sourceNamedEntities = ner(sourceParseResult)
    sourceNamedEntities = sorted(sourceNamedEntities, key=len)
    
    targetNamedEntities = ner(targetParseResult)
    targetNamedEntities = sorted(targetNamedEntities, key=len)
    

    # learn from the other sentence that a certain word/phrase is a named entity (learn for source from target)
    for item in source:
        alreadyIncluded = False
        for jtem in sourceNamedEntities:
            if item[1] in jtem[1]:
                alreadyIncluded = True
                break
        if alreadyIncluded or (len(item[2]) >0 and not item[2][0].isupper()):
            continue
        for jtem in targetNamedEntities:
            if item[2] in jtem[2]:
                # construct the item
                newItem = [[item[0]], [item[1]], [item[2]], jtem[3]]

                # check if the current item is part of a named entity part of which has already been added (by checking contiguousness)
                partOfABiggerName = False
                for k in xrange(len(sourceNamedEntities)):
                    if sourceNamedEntities[k][1][len(sourceNamedEntities[k][1])-1] == newItem[1][0] - 1:
                        sourceNamedEntities[k][0].append(newItem[0][0])
                        sourceNamedEntities[k][1].append(newItem[1][0])
                        sourceNamedEntities[k][2].append(newItem[2][0])
                        partOfABiggerName = True
                if not partOfABiggerName:
                    sourceNamedEntities.append(newItem)
            elif isAcronym(item[2], jtem[2]) and [[item[0]], [item[1]], [item[2]], jtem[3]] not in sourceNamedEntities:
                sourceNamedEntities.append([[item[0]], [item[1]], [item[2]], jtem[3]])


    
    # learn from the other sentence that a certain word/phrase is a named entity (learn for target from source)
    for item in target:
        alreadyIncluded = False
        for jtem in targetNamedEntities:
            if item[1] in jtem[1]:
                alreadyIncluded = True
                break
        if alreadyIncluded or (len(item[2]) >0 and not item[2][0].isupper()):
            continue
        for jtem in sourceNamedEntities:
            if item[2] in jtem[2]:
                # construct the item
                newItem = [[item[0]], [item[1]], [item[2]], jtem[3]]

                # check if the current item is part of a named entity part of which has already been added (by checking contiguousness)
                partOfABiggerName = False
                for k in xrange(len(targetNamedEntities)):
                    if targetNamedEntities[k][1][len(targetNamedEntities[k][1])-1] == newItem[1][0] - 1:
                        targetNamedEntities[k][0].append(newItem[0][0])
                        targetNamedEntities[k][1].append(newItem[1][0])
                        targetNamedEntities[k][2].append(newItem[2][0])
                        partOfABiggerName = True
                if not partOfABiggerName:
                    targetNamedEntities.append(newItem)
            elif isAcronym(item[2], jtem[2]) and [[item[0]], [item[1]], [item[2]], jtem[3]] not in targetNamedEntities:
                targetNamedEntities.append([[item[0]], [item[1]], [item[2]], jtem[3]])
    

    sourceWords = []
    targetWords = []

    for item in sourceNamedEntities:
        for jtem in item[1]:
            if item[3] in ['PERSON', 'ORGANIZATION', 'LOCATION']:
                sourceWords.append(source[jtem-1][2])
    for item in targetNamedEntities:
        for jtem in item[1]:
            if item[3] in ['PERSON', 'ORGANIZATION', 'LOCATION']:
                targetWords.append(target[jtem-1][2])



    if len(sourceNamedEntities) == 0 or len(targetNamedEntities) == 0:
        return []




    sourceNamedEntitiesAlreadyAligned = []
    targetNamedEntitiesAlreadyAligned = []
    

    # align all full matches
    for item in sourceNamedEntities:
        if item[3] not in ['PERSON', 'ORGANIZATION', 'LOCATION']:
            continue

        
        # do not align if the current source entity is present more than once
        count = 0
        for ktem in sourceNamedEntities:
            if ktem[2] == item[2]:
                count += 1
        if count > 1:
            continue


        for jtem in targetNamedEntities:
            if jtem[3] not in ['PERSON', 'ORGANIZATION', 'LOCATION']:
                continue

            # do not align if the current target entity is present more than once
            count = 0
            for ktem in targetNamedEntities:
                if ktem[2] == jtem[2]:
                    count += 1
            if count > 1:
                continue


            # get rid of dots and hyphens
            canonicalItemWord = [i.replace('.', '') for i in item[2]]
            canonicalItemWord = [i.replace('-', '') for i in item[2]]
            canonicalJtemWord = [j.replace('.', '') for j in jtem[2]]
            canonicalJtemWord = [j.replace('-', '') for j in jtem[2]]

            if canonicalItemWord == canonicalJtemWord:
                for k in xrange(len(item[1])):
                    if ([item[1][k], jtem[1][k]]) not in alignments:
                        alignments.append([item[1][k], jtem[1][k]])
                sourceNamedEntitiesAlreadyAligned.append(item)
                targetNamedEntitiesAlreadyAligned.append(jtem)

    # align acronyms with their elaborations
    for item in sourceNamedEntities:
        if item[3] not in ['PERSON', 'ORGANIZATION', 'LOCATION']:
            continue
        for jtem in targetNamedEntities:
            if jtem[3] not in ['PERSON', 'ORGANIZATION', 'LOCATION']:
                continue

            if len(item[2])==1 and isAcronym(item[2][0], jtem[2]):
                for i in xrange(len(jtem[1])):
                    if [item[1][0], jtem[1][i]] not in alignments:
                        alignments.append([item[1][0], jtem[1][i]])
                        sourceNamedEntitiesAlreadyAligned.append(item[1][0])
                        targetNamedEntitiesAlreadyAligned.append(jtem[1][i])

            elif len(jtem[2])==1 and isAcronym(jtem[2][0], item[2]):
                for i in xrange(len(item[1])):
                    if [item[1][i], jtem[1][0]] not in alignments:
                        alignments.append([item[1][i], jtem[1][0]])
                        sourceNamedEntitiesAlreadyAligned.append(item[1][i])
                        targetNamedEntitiesAlreadyAligned.append(jtem[1][0])

                               
    # align subset matches
    for item in sourceNamedEntities:
        if item[3] not in ['PERSON', 'ORGANIZATION', 'LOCATION'] or item in sourceNamedEntitiesAlreadyAligned:
            continue

        # do not align if the current source entity is present more than once
        count = 0
        for ktem in sourceNamedEntities:
            if ktem[2] == item[2]:
                count += 1
        if count > 1:
            continue


        for jtem in targetNamedEntities:
            if jtem[3] not in ['PERSON', 'ORGANIZATION', 'LOCATION'] or jtem in targetNamedEntitiesAlreadyAligned:
                continue

            if item[3] <> jtem[3]:
                continue

            # do not align if the current target entity is present more than once
            count = 0
            for ktem in targetNamedEntities:
                if ktem[2] == jtem[2]:
                    count += 1
            if count > 1:
                continue

            
            # find if the first is a part of the second
            if isSublist(item[2], jtem[2]):
                unalignedWordIndicesInTheLongerName = []
                for ktem in jtem[1]:
                    unalignedWordIndicesInTheLongerName.append(ktem)
                for k in xrange(len(item[2])):
                    for l in xrange(len(jtem[2])):
                        if item[2][k] == jtem[2][l] and [item[1][k], jtem[1][l]] not in alignments:
                            alignments.append([item[1][k], jtem[1][l]])
                            if jtem[1][l] in unalignedWordIndicesInTheLongerName:
                                unalignedWordIndicesInTheLongerName.remove(jtem[1][l])
                for k in xrange(len(item[1])): # the shorter name
                    for l in xrange(len(jtem[1])): # the longer name
                        # find if the current term in the longer name has already been aligned (before calling alignNamedEntities()), do not align it in that case
                        alreadyInserted = False
                        for mtem in existingAlignments:
                            if mtem[1] == jtem[1][l]:
                                alreadyInserted = True
                                break
                        if jtem[1][l] not in unalignedWordIndicesInTheLongerName or alreadyInserted:
                            continue
                        if [item[1][k], jtem[1][l]] not in alignments  and target[jtem[1][l]-1][2] not in sourceWords  and item[2][k] not in punctuations and jtem[2][l] not in punctuations:
                            alignments.append([item[1][k], jtem[1][l]])
                
            # else find if the second is a part of the first
            elif isSublist(jtem[2], item[2]):
                unalignedWordIndicesInTheLongerName = []
                for ktem in item[1]:
                    unalignedWordIndicesInTheLongerName.append(ktem)
                for k in xrange(len(jtem[2])):
                    for l in xrange(len(item[2])):
                        if jtem[2][k] == item[2][l] and [item[1][l], jtem[1][k]] not in alignments:
                            alignments.append([item[1][l], jtem[1][k]])
                            if item[1][l] in unalignedWordIndicesInTheLongerName:
                                unalignedWordIndicesInTheLongerName.remove(item[1][l])
                for k in xrange(len(jtem[1])): # the shorter name
                    for l in xrange(len(item[1])): # the longer name
                        # find if the current term in the longer name has already been aligned (before calling alignNamedEntities()), do not align it in that case
                        alreadyInserted = False
                        for mtem in existingAlignments:
                            if mtem[0] == item[1][k]:
                                alreadyInserted = True
                                break
                        if item[1][l] not in unalignedWordIndicesInTheLongerName or alreadyInserted:
                            continue
                        if [item[1][l], jtem[1][k]] not in alignments  and source[item[1][k]-1][2] not in targetWords  and item[2][l] not in punctuations and jtem[2][k] not in punctuations:
                            alignments.append([item[1][l], jtem[1][k]])
                            #unalignedWordIndicesInTheLongerName.remove(jtem[1][l])

                

    return alignments

def isAcronym(word, namedEntity):
# returns whether 'word' is an acronym of 'namedEntity', which is a list of the component words
    canonicalWord = word.replace('.', '')
    if not canonicalWord.isupper() or len(canonicalWord) <> len(namedEntity) or canonicalWord.lower() in ['a', 'i']:
        return False

    acronym = True    
    for i in xrange(len(canonicalWord)):
        if canonicalWord[i] <> namedEntity[i][0]:
            acronym = False
            break

    return acronym

In [227]:
print alignments
print alignNamedEntities(sentence1LemmasAndPosTags, sentence2LemmasAndPosTags, sentence1ParseResult, sentence2ParseResult, alignments)

[[7, 8], [8, 9]]
[]
